In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
from fetch_data import fetch_data_groups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models

import sys
sys.path.append('..')
from utils import preprocess, get_windows

F:\InstallationFiles\Anaconda3\envs\for_pytorch\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [36]:
MIN_COUNTS = 20
MAX_COUNTS = 1800
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 15
# minimum document length 
# (number of words)
# after preprocessing

# half the size of the context around a word
HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model

In [37]:
nlp = spacy.load('en')

# Load dataset

In [57]:
from fetch_data import fetch_data_groups
dataset = fetch_data_groups(data_home='D:/code python/LDA2vec/bangla news/data')
docs = dataset['data']

In [58]:
# number of documents
len(docs)

67

In [59]:
# store an index with a document
docs = [(i, doc) for i, doc in enumerate(docs)]

In [ ]:
print(docs[0])

# Preprocess dataset and create windows

In [60]:
from utils import preprocess, get_windows
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)

 58%|███████████████████████████████████████████████▋                                  | 39/67 [00:09<00:07,  4.00it/s]F:\InstallationFiles\Anaconda3\envs\for_pytorch\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████████████████████████████████████████████████████████████████████████████| 67/67 [00:17<00:00,  3.77it/s]


number of removed short documents: 1
total number of tokens: 9640
number of tokens to be removed: 8778
number of additionally removed short documents: 41
total number of tokens: 555

minimum word count number: 5
this number can be less than MIN_COUNTS because of document removal


In [61]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [62]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')

25it [00:00, ?it/s]


In [63]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [64]:
# number of windows (equals to the total number of tokens)
data.shape[0]

555

# Get unigram distribution

In [65]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

# Prepare word vectors

In [73]:
%%time
vocab_size = len(decoder)
embedding_dim = 50

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=embedding_dim, window=5, workers=4, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
    word_vectors[i] = model.wv[str(i)]

In [67]:
# number of unique words
vocab_size

31

# Prepare initialization for document weights

In [68]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [69]:
%%time
n_topics = 2
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

Wall time: 142 ms


In [70]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : সময় ঢাকা বাংলাদেশ দ্য ছবিতে বেলা কথা ছবির মার্চ সকাল
topic 1 : ছবির বাংলাদেশ জানান ছবি সময় শেষ নাটক হয়েছে দেশের সময়


In [71]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

100%|█████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 370.63it/s]


# Save data

In [72]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)

In [46]:
np.save('doc_weights_init.npy', doc_weights_init)